In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
#!unzip -qq /content/drive/'My Drive'/Code/tomatoes.zip

In [2]:
import cv2
import os
from tqdm.notebook import tqdm
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow.keras.layers as L
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

In [3]:
dataset_path = '/content/PlantVillage'

class_name = []
images = []
labels = []
for index, folder in enumerate(os.listdir(dataset_path)):
  class_name.append(folder)
  image_path = os.path.join(dataset_path, folder)
  # print(image_path, len(os.listdir(image_path)))
  for image in os.listdir(image_path):
    labels.append(index)
    images.append(os.path.join(dataset_path, folder, image))

X = []
for index, path in enumerate(tqdm(images)):
  try:
    X.append(cv2.resize(cv2.imread(path),dsize=(256,256)))
  except:
    labels.pop(index)
X = np.array(X)
y = np.array(labels)
Y = to_categorical(y, num_classes=len(class_name))
Y = to_categorical(y, num_classes=len(class_name))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4,stratify = Y, shuffle=True, random_state=100)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify = y_test, shuffle=True, random_state=100)
print(X_train.shape, X_test.shape,X_val.shape)
print(y_train.shape, y_test.shape,y_val.shape)

(7426, 256, 256, 3) (2476, 256, 256, 3) (2475, 256, 256, 3)
(7426, 5) (2476, 5) (2475, 5)


In [5]:
def get_model():
    model = InceptionResNetV2(
            input_shape=(256, 256, 3),
            weights='imagenet',
            include_top=False
        )
    for layer in model.layers:
        layer.trainable = False 

    input = Input(shape=(256, 256, 3))
    x = Conv2D(3, (3, 3), padding='same')(input)
    x = model(x)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(5, activation='softmax', name='predictions')(x)
    # multi output
    output = Dense(10,activation = 'softmax', name='root')(x)

    # Compile
    model = Model(input, x)
    optimizer = Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    

    return model

inceptionv3model = get_model()

filepath="/content/weights-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.1,
    rescale=1./255,
	width_shift_range=0.1,
    height_shift_range=0.1,
	horizontal_flip=True,
    brightness_range=[0.2,1.5], fill_mode="nearest")

aug_val = ImageDataGenerator(rescale=1./255)

history=inceptionv3model.fit(aug.flow(X_train, y_train, batch_size=32),
                               epochs=25, steps_per_epoch=len(X_train)//32,
                               validation_data=aug.flow(X_val,y_val,
                               batch_size=64), callbacks=callbacks_list)

inceptionv3model.save("inceptionv3model.h5")


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_203 (Conv2D)          (None, 256, 256, 3)       84        
_________________________________________________________________
inception_resnet_v2 (Functio (None, 6, 6, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
batch_normalization_203 (Bat (None, 1536)              6144      
_________________________________________________________________
dropout (Dropout)            (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 32)               

In [7]:
from keras.models import load_model
#for i in range(1,10):
#  modellink = '/content/weights-' + '0' + str(i) + '.hdf5'
#  model = load_model(modellink)
#  scores = model.evaluate(X_test, y_test)
#  print(f"Test Accuracy: {scores[1]*100}")
#for i in range(10,26):
#  modellink = '/content/weights-' + str(i) + '.hdf5'
#  model = load_model(modellink)
#  scores = model.evaluate(X_test, y_test)
#  print(f"Test Accuracy: {scores[1]*100}")
model = load_model(r'/content/weights-17.hdf5')
scores = model.evaluate(X_test, y_test)

78/78 [==============================] - 10s 129ms/step - loss: 796.9929 - accuracy: 0.2019
